# Building a trust-pilot scrapper

## Imports

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime as dt

/Users/kaushikdayalan/projects/storymachine_projects/trustpilot_scraper/.env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
## Initialize lists
review_titles = []
review_dates_original = []
review_dates = []
review_ratings = []
review_texts = []
page_number = []
replies = []

In [ ]:
web_page

In [79]:
response = requests.get(f"https://de.trustpilot.com/review/{website_name}?page={i}")
    
web_page = response.text
soup = BeautifulSoup(web_page, "html.parser")



{'id': '653f65da2b18c6e12a6ed0a3',
 'filtered': False,
 'pending': False,
 'text': 'Weniger als 1* Wert.\nIch habe lange ignoriert, dass die Zusteller von DHL keine Lust haben zu arbeiten, zu Faul oder zu unfähig sind. \nHeute aber die Krönung… Ich habe ein Paket am Freitag erwartet und es war auch in Zustellung. Die Tour wurde abgebrochen und das Paket “wie gewünscht” in die Filiale gebracht. Welche Filiale? Keine Ahnung. Heute (Montag) habe ich bei DHL angerufen und da bestätigt man mir die korrekte Adresse, sagt mir aber, dass der Zusteller die Hausnummer nicht gefunden hat und das Paket deswegen zurückgeschickt wird. Zurückgeschickt. Warum?\nJetzt ist Halloween versaut und mein Kind bitterlich am weinen',
 'rating': 1,
 'labels': {'merged': None,
  'verification': {'isVerified': False,
   'createdDateTime': '2023-10-30T10:14:18.000Z',
   'reviewSourceName': 'Organic',
   'verificationSource': 'invitation',
   'verificationLevel': 'not-verified',
   'hasDachExclusion': False}},
 'ti

In [ ]:
# Set Trustpilot page numbers to scrape here
from_page = 1
to_page = 500

df_data = []

website_name = "www.dhl.de"

for i in range(from_page, to_page + 1):
    response = requests.get(f"https://de.trustpilot.com/review/{website_name}?page={i}")
    
    web_page = response.text
    soup = BeautifulSoup(web_page, "html.parser")
    data = json.loads(soup.find('script' , id="__NEXT_DATA__",type='application/json').text)
    data = data["props"]["pageProps"]["reviews"]
    for j, review in enumerate(data):
        reviews = []
        reviews.extend([
            review["consumer"]["displayName"],
                        review["consumer"]["countryCode"],
                        review["dates"]["publishedDate"],
                        review["title"], 
                        review["text"], 
                        review["rating"],
                        review["likes"]])
        
        if review["reply"] != None:
            reviews.append(review["reply"]["message"])
        else:
            reviews.append(None)
        df_data.append(reviews)
        break

In [ ]:
df_columns = ["reviewerName", 
              "reviewerCountryCode",
              "postDate",
              "postTitle", 
              "postContent", 
              "rating",
              "likes",
              "reply"
             ]

df = pd.DataFrame(data=df_data, columns=df_columns)

In [ ]:
df.shape